<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Delivery%20Tour%20Berlin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP - Nächster Nachbar Heuristik

## Daten

In [ ]:
!pip install haversine
import haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data = dict()
data["Halle (Saale)"]=dict(lon=11.9672201241301, lat=51.4970124613279)
data["Magdeburg"]=dict(lon=11.6363317198121, lat=52.1229002894915)
data["Dessau-Roßlau"]=dict(lon=12.2331141970666, lat=51.8442730013004)
data["Lutherstadt Wittenberg"]=dict(lon=12.6357720180488, lat=51.8737704942318)
data["Weißenfels"]=dict(lon=11.9683230903702, lat=51.1983573259)
data["Halberstadt"]=dict(lon=11.0495250643003, lat=51.8912329116255)
data["Stendal"]=dict(lon=11.8508504963119, lat=52.602278283988)

# Direktdistanz berechnen
dist = {i: {j: haversine.haversine((data[i]['lat'], data[i]['lon']), (data[j]['lat'], data[j]['lon']))
            for j in data}
        for i in data}

print(dist)

{'Halle (Saale)': {'lon': 11.9672201241301, 'lat': 51.4970124613279}, 'Magdeburg': {'lon': 11.6363317198121, 'lat': 52.1229002894915}, 'Dessau-Roßlau': {'lon': 12.2331141970666, 'lat': 51.8442730013004}, 'Lutherstadt Wittenberg': {'lon': 12.6357720180488, 'lat': 51.8737704942318}, 'Weißenfels': {'lon': 11.9683230903702, 'lat': 51.1983573259}, 'Halberstadt': {'lon': 11.0495250643003, 'lat': 51.8912329116255}, 'Stendal': {'lon': 11.8508504963119, 'lat': 52.602278283988}}
{'Halle (Saale)': {'Halle (Saale)': 0.0, 'Magdeburg': 73.21886807930814, 'Dessau-Roßlau': 42.74611270360542, 'Lutherstadt Wittenberg': 62.283370366529105, 'Weißenfels': 33.2090700943156, 'Halberstadt': 76.95626197830808, 'Stendal': 123.15742997802015}, 'Magdeburg': {'Halle (Saale)': 73.21886807930814, 'Magdeburg': 0.0, 'Dessau-Roßlau': 51.285449187848485, 'Lutherstadt Wittenberg': 73.81705287547693, 'Weißenfels': 105.32385322531576, 'Halberstadt': 47.71628122394498, 'Stendal': 55.25883804513558}, 'Dessau-Roßlau': {'Halle

## Implementierung

### Daten

In [ ]:
# Angebotsknoten
A = ['Sunnyvale', 'Dublin', 'Bankok']

# Angebotsmenge
a = {'Sunnyvale': 45, 'Dublin': 120, 'Bankok': 95}

# Nachfrageknoten
N = ['Amarillo', 'Teaneck', 'Chicago', 'Falls']

# Nachfragemenge
b = {'Amarillo': 80, 'Teaneck': 78, 'Chicago': 47, 'Falls': 55}

# Kosten
c = {
    'Sunnyvale': {'Amarillo': 250, 'Teaneck': 420, 'Chicago': 380, 'Falls': 280},
    'Dublin': {'Amarillo': 1280, 'Teaneck': 990, 'Chicago': 1440, 'Falls': 1520},
    'Bankok': {'Amarillo': 550, 'Teaneck': 1420, 'Chicago': 1660, 'Falls': 1730},
}

### Modell

In [ ]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 54.6 MB/s eta 0:00:00


In [ ]:
# Model
model = pulp.LpProblem(name='Transportproblem',
                       sense=pulp.constants.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts(name='x', indices=(A, N), lowBound=0, cat='Integer')

# (1) Objective
model += pulp.lpSum(c[i][j] * x[i][j] for i in A for j in N), '(1)'

# (2)
for i in A:
  model += a[i] == pulp.lpSum(x[i][j] for j in N), '(2)_' + str(i)

# (2)
for j in N:
  model += b[j] == pulp.lpSum(x[i][j] for i in A), '(3)_' + str(j)

# print(model)

### Solution

In [ ]:
# solve problem
model.solve()

# get status
print("Status:", pulp.LpStatus[model.status])

# get objective value
print('Objective value:', round(pulp.value(model.objective), 2))

# get value of decision variable u (position in tour of the nodes that are part of the tour)
for i in A:
  for j in N:
    if x[i][j].varValue > 0:
      print('Von ' + str(i) + ' nach ' + str(j) + ': ' + str(x[i][j].varValue) + ' ME')

Status: Optimal
Objective value: 219900.0
Von Sunnyvale nach Falls: 45.0 ME
Von Dublin nach Teaneck: 78.0 ME
Von Dublin nach Chicago: 42.0 ME
Von Bankok nach Amarillo: 80.0 ME
Von Bankok nach Chicago: 5.0 ME
Von Bankok nach Falls: 10.0 ME
